In [2]:
import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import apply_affine_transform
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import RMSprop, Adam
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.callbacks import ReduceLROnPlateau

from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight


from PIL import Image
import matplotlib.pyplot as plt

from collections import Counter


# Load Dataset

In [3]:
CSV_FILE_NAME = './data/my_file.csv'
# CSV_FILE_NAME = './data/my_file_400.csv'
arr = pd.read_csv(CSV_FILE_NAME, header=None)

In [4]:
X, y_class, y_reg = arr.iloc[:, :-5].values, arr.iloc[:, -5:-4].values, arr.iloc[:, -4:].values
# X = X.reshape(-1, 200, 200, 1)
# X = X.reshape(-1, 400, 400, 1)


# one hot encoding
y_class = tf.keras.utils.to_categorical(y_class, num_classes=17)

# dataset split
X_train, X_test, y_train_class, y_test_class = train_test_split(X, y_class, test_size=0.2, random_state=42, stratify=y_class)


# if no data augmentation
class_weights = class_weight.compute_class_weight(class_weight='balanced',
                                                  classes=np.unique(y_train_class.argmax(axis=1)),
                                                  y=y_train_class.argmax(axis=1))

class_weights_dict = dict(enumerate(class_weights))


# Model

In [5]:
model = models.Sequential([
    layers.Input(shape=(40000,)),
    # layers.Dense(40000, activation='relu'),
    layers.Dense(5000, activation='relu'),
    layers.Dense(1000, activation='relu'),
    layers.Dense(100, activation='relu'),
    layers.Dense(50, activation='relu'),
    layers.Dense(25, activation='relu'),
    layers.Dense(17, activation=None),
])


reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=np.sqrt(0.1), patience=5)

# f1_metric = tf.metrics.F1Score(average='macro')
f1_metric = tf.metrics.F1Score(average='weighted') # if no data augmentation


model.compile(optimizer=Adam(0.001),
              loss=CategoricalCrossentropy(),
              metrics=['accuracy', f1_metric])


model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 40000)          │ 1,600,040,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 5000)           │   200,005,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1000)           │     5,001,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 100)            │       100,100 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 50)             │         5,050 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 25)             │         1,275 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 17)             │           442 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,805,152,867 (6.72 GB)

 Trainable params: 1,805,152,867 (6.72 GB)

 Non-trainable params: 0 (0.00 B)

In [6]:
# history = model.fit(X_balanced, y_balanced, epochs=20, validation_data=(X_test, y_test_class))
history = model.fit(X_train, y_train_class, epochs=50, validation_data=(X_test, y_test_class)) # if no data augmentation

Epoch 1/50


: 

In [ ]:
# Convert model history dictionary to DataFrame
df_loss_acc = pd.DataFrame(model.history.history)

# Select loss columns and rename
df_loss = df_loss_acc[['loss', 'val_loss']].copy()
df_loss.rename(columns={'loss': 'train', 'val_loss': 'validation'}, inplace=True)

# Select accuracy columns and rename
df_acc = df_loss_acc[['accuracy', 'val_accuracy']].copy()
df_acc.rename(columns={'accuracy': 'train', 'val_accuracy': 'validation'}, inplace=True)

# Plot model loss
df_loss.plot(title='Model loss', figsize=(12, 8)).set(xlabel='Epoch', ylabel='Loss')

# Plot model accuracy
df_acc.plot(title='Model Accuracy', figsize=(12, 8)).set(xlabel='Epoch', ylabel='Accuracy')


In [ ]:
result = model.evaluate(X_test, y_test_class)
print("Result:", result)

In [ ]:
model = models.Sequential([
    layers.Input(shape=(200, 200, 1)),
    layers.Conv2D(32, 3, activation='relu'),
    layers.MaxPooling2D(2),
    layers.Conv2D(64, 3, activation='relu'),
    layers.MaxPooling2D(2),
    layers.Flatten(),
    layers.Dropout(0.2),
    layers.Dense(64, activation='relu'),
    layers.Dropout(0.2),
    layers.Dense(17, activation='softmax'),
])


# f1_metric = tf.metrics.F1Score(average='macro')
f1_metric = tf.metrics.F1Score(average='weighted') # if no data augmentation

model.compile(optimizer=Adam(learning_rate=0.00001),
              loss=CategoricalCrossentropy(),
              metrics=['accuracy', f1_metric])


model.summary()


In [ ]:
history2 = model.fit(X_train, y_train_class, epochs=50, validation_data=(X_test, y_test_class), class_weight=class_weights_dict) # if no data augmentation

In [ ]:
history2_continued = model.fit(X_train, y_train_class, epochs=100, initial_epoch=50, validation_data=(X_test, y_test_class), class_weight=class_weights_dict) # if no data augmentation